In [46]:
import matplotlib.pyplot as plt
import numpy as np
import torch as t
import torch.nn as nn
import pandas as pd
import warnings
import math
import itertools
import copy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta
warnings.filterwarnings("ignore")

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Data preprocessing

In [48]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/43project4/compas-scores-two-years-violent.csv")
data

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,two_year_recid.1
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,1
2,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0,0
3,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,2013-03-26,NaN,NaN,2,0,1102,0,0,0
4,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,10995,raheem smith,raheem,smith,2013-10-20,Male,1995-06-28,20,Less than 25,African-American,...,High,2013-10-20,2014-04-07,2014-04-27,0,0,169,0,0,0
4739,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0,0
4740,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0,0
4741,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0,0


In [49]:
df = data[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text', 'sex', 'priors_count',
           'decile_score', 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out','is_violent_recid']]


df = df.loc[df['race'].isin(('African-American', 'Caucasian'))]
df.loc[df["race"] == "African-American", "race"] = 0
df.loc[df["race"] == "Caucasian", "race"] = 1

df = df.replace({'sex': 'Male'}, 1)
df = df.replace({'sex': 'Female'}, 0)

df = df.loc[df['is_recid'] != -1]
df = df.loc[df['c_charge_degree'] != 'O']
df = df.loc[df['score_text'] != 'N/A']

df['length_of_stay'] = (df['c_jail_out'].apply(pd.to_datetime) - df['c_jail_in'].apply(pd.to_datetime)).dt.days
df = df.dropna(subset = ['length_of_stay'])
df['length_of_stay'] = df['length_of_stay'].apply(lambda length_of_stay: 0 if length_of_stay <= 7 else (2 if length_of_stay > 90 else 1))
df = df.drop(columns=['c_jail_in', 'c_jail_out'])

df['priors_count'] = df['priors_count'].apply(lambda priors_count: 0 if priors_count == 0 else (2 if priors_count > 3 else 1))

df = df.replace({'age_cat': 'Less than 25'}, 0)
df = df.replace({'age_cat': '25 - 45'}, 1)
df = df.replace({'age_cat': 'Greater than 45'}, 2)

df = df.replace({'c_charge_degree': 'F'}, 0)
df = df.replace({'c_charge_degree': 'M'}, 1)

df = df.replace({'score_text': 'Low'}, 0)
df = df.replace({'score_text': 'Medium'}, 1)
df = df.replace({'score_text': 'High'}, 2)

df = df.drop_duplicates()

df.tail()

,age,c_charge_degree,race,age_cat,score_text,sex,priors_count,decile_score,is_recid,two_year_recid,is_violent_recid,length_of_stay
4732,32,0,0,1,1.0,1,2,5,0,0,0,0
4733,23,0,1,0,2.0,1,0,8,0,0,0,0
4737,23,1,1,0,2.0,1,2,10,1,1,1,1
4738,20,0,0,0,2.0,1,0,9,0,0,0,0
4742,33,1,0,1,0.0,0,1,2,0,0,0,0


## 2. Split dataset into 5:1:1

In [50]:
X = df.drop(columns=["two_year_recid"])
Y = df["two_year_recid"]

#split dataset so that training:validation:testing=5:1:1
df_X_train, df_X_rem, df_Y_train, df_Y_rem = train_test_split(X,Y, train_size=5/7.0)
df_X_valid, df_X_test, df_Y_valid, df_Y_test = train_test_split(df_X_rem, df_Y_rem, test_size = 0.5)

A7_df = df

label = "two_year_recid"
sensitive = "race"
features = ['c_charge_degree', 'age_cat', 'sex' ,'priors_count', 'length_of_stay']
features_race = ['race', 'c_charge_degree', 'age_cat', 'sex','priors_count', 'length_of_stay']

train_A7 = A7_df[:int(len(A7_df) * (5/7))]
test_A7 = A7_df[int(len(A7_df) * (5/7)):int(len(A7_df) * (6/7))]
vali_A7 = A7_df[int(len(A7_df) * (6/7)):]

x_train1 = train_A7[features]
y_train1 = train_A7[label].to_numpy()
race_train1 = train_A7[sensitive]

x_test1 = test_A7[features]
y_test1 = test_A7[label].to_numpy()
race_test1 = test_A7[sensitive]

x_validation1 = vali_A7[features]
y_validation1 = vali_A7[label].to_numpy()
race_validation1 = vali_A7[sensitive]

x_train_race = train_A7[features_race]
x_test_race = test_A7[features_race]
x_validation_race = vali_A7[features_race]

x_validation1.head()

,c_charge_degree,age_cat,sex,priors_count,length_of_stay
3969,1,1,0,1,0
3974,1,1,1,1,0
3977,1,0,1,0,0
3979,1,1,1,2,0
3982,0,1,1,1,0


## 3. Baseline Model

Baseline model with the featrue race

In [51]:
clf_race = LogisticRegression().fit(x_train_race, y_train1)
accuracy_race = clf_race.score(x_validation_race, y_validation1)
accuracy_race

0.7919621749408984

Baseline model without the featrue race

In [52]:
clf_withtout_race = LogisticRegression().fit(x_train1, y_train1)
accuracy_withtout_race = clf_withtout_race.score(x_validation1, y_validation1)
accuracy_withtout_race

0.7872340425531915

## 4. Funtion from 'Information Theoretic Measures for Fairness-aware Feature Selection'

In [53]:
def uni_values_array(arr):

    # arr: n * m matrix
    # each elements in uni_values_array gives unique values from the matrix

    uni_values = []
    for col in range(arr.shape[1]):
        uni_values.append(np.unique(arr[:, col]).tolist())
    return uni_values

def power_func(seq):

    #This function create a generator that contains all subsets of seq

    if len(seq) <= 1:
        yield seq
        yield []
    else:
        for i in power_func(seq[1:]):
            yield [seq[0]] + i
            yield i

def unique_information(array_1, array_2):
    assert array_1.shape[0] == array_2.shape[0]

    n_rows = array_1.shape[0]
    n_col_array_1 = array_1.shape[1]

    concated_array = np.concatenate((array_1, array_2), axis=1)
    unique_array = uni_values_array(concated_array)
    cartesian_product = list(itertools.product(*unique_array))

    # IQ(T; R1|R2) = ∑t,r1,r2 QT ,R1,R2 (t, r1, r2) log((QT |R1,R2 (t|r1,r2))/ (QT |R2 (t|r2)))

    IQ = 0
    for i in cartesian_product:
        r1_r2 = len(np.where((concated_array == i).all(axis=1))[0]) / n_rows
        r1 = len(np.where((array_1 == i[:n_col_array_1]).all(axis=1))[0]) / n_rows
        r2 = len(np.where((array_2 == i[n_col_array_1:]).all(axis=1))[0]) / n_rows

        if r1_r2 == 0 or r1 == 0 or r2 == 0:
            IQ_iter = 0
        else:
            IQ_iter = r1_r2 * np.log(r1_r2 / r1) / r1
        IQ += np.abs(IQ_iter)
    return IQ


def unique_infor_condi(array_1, array_2, conditional):
    assert (array_1.shape[0] == array_2.shape[0]) and (array_1.shape[0] == conditional.shape[0])

    n_rows = array_1.shape[0]
    n_col_array_1 = array_1.shape[1]
    n_col_array_2 = array_2.shape[1]

    concated_array_2_conditional = np.concatenate((array_2, conditional), axis=1)
    concated_array_all           = np.concatenate((array_1, concated_array_2_conditional), axis=1)
    unique_array                 = uni_values_array(concated_array_all)
    cartesian_product = list(itertools.product(*unique_array))

    IQ = 0
    for i in cartesian_product:
        r1_r2 = len(np.where((concated_array_all == i).all(axis=1))[0]) / n_rows
        r1    = len(np.where((array_1 == i[:n_col_array_1]).all(axis=1))[0]) / n_rows
        r2    = len(np.where((concated_array_all[:, n_col_array_1: -n_col_array_2] == i[n_col_array_1: -n_col_array_2]).all(axis=1))[0]) / n_rows

        try:
            r1_given_r2 = len(np.where((concated_array_all[:, :n_col_array_1] == i[ :n_col_array_1]).all(axis=1) & (concated_array_all[:, -n_col_array_2:] == i[-n_col_array_2:]).all(axis=1))[0]) / len(np.where((concated_array_all[:, -n_col_array_2:] == i[-n_col_array_2:]).all(axis=1))[0])
        except ZeroDivisionError:
            r1_given_r2 = 0

        if r1_r2 == 0 or r1 == 0 or r2 == 0 or r1_given_r2 == 0:
            IQ_iter = 0
        else:
            IQ_iter = r1_r2 * np.log(r1_r2 / r2) / r1_given_r2
        IQ += np.abs(IQ_iter)
    return IQ

def accuracy_coef(y, x_s, x_s_c, f_num):
    conditional = np.concatenate((x_s_c, f_num), axis=1)
    return unique_infor_condi(y, x_s, conditional)

def discrimination_coef(y, x_s, f_num):
    x_s_a = np.concatenate((x_s, f_num), axis=1)
    return unique_information(y, x_s_a) * unique_information(x_s, f_num) * unique_infor_condi(x_s, f_num, y)

def marginal_accuracy_coef(y_train, x_train, f_num, set_tracker):

    n_features = x_train.shape[1]
    feature_idx = list(range(n_features))
    feature_idx.pop(set_tracker)
    power_func_features = [x for x in power_func(feature_idx) if len(x) > 0]

    shapley_value =0
    for sc_idx in power_func_features:
            coef = math.factorial(len(sc_idx)) * math.factorial(n_features - len(sc_idx) - 1) / math.factorial(n_features)

            # Compute v(T ∪ {i})
            idx_xs_ui = copy.copy(sc_idx)
            idx_xs_ui.append(set_tracker)
            idx_xsc_ui = list(set(list(range(n_features))).difference(set(idx_xs_ui))) # compliment of x_s
            vTU = accuracy_coef(y_train.reshape(-1, 1), x_train[:, idx_xs_ui], x_train[:, idx_xsc_ui], f_num.reshape(-1, 1))

             # Compute v(T)
            idx_xsc = list(range(n_features))
            idx_xsc.pop(set_tracker)
            idx_xsc = list(set(idx_xsc).difference(set(sc_idx)))
            vT = accuracy_coef(y_train.reshape(-1, 1), x_train[:, sc_idx], x_train[:, idx_xsc], f_num.reshape(-1, 1))

            marginal = vTU - vT
            shapley_value = shapley_value + coef * marginal
    return shapley_value

def marginal_discrimination_coef(y_train, x_train, f_num, set_tracker):

    n_features = x_train.shape[1]
    feature_idx = list(range(n_features))
    feature_idx.pop(set_tracker)
    power_func_features = [x for x in power_func(feature_idx) if len(x) > 0]

    shapley_value =0
    for sc_idx in power_func_features:
            coef = math.factorial(len(sc_idx)) * math.factorial(n_features - len(sc_idx) - 1) / math.factorial(n_features)

            # Compute v(T ∪ {i})
            idx_xs_ui = copy.copy(sc_idx)
            idx_xs_ui.append(set_tracker)
            vTU = discrimination_coef(y_train.reshape(-1, 1), x_train[:, idx_xs_ui], f_num.reshape(-1, 1))

            # Compute v(T)
            vT = discrimination_coef(y_train.reshape(-1, 1), x_train[:, sc_idx], f_num.reshape(-1, 1))

            marginal = vTU - vT
            shapley_value = shapley_value + coef * marginal

    return shapley_value

##  5. Shapley Value

In [54]:
shapley_acc = []
shapley_disc = []
x_train1_np = x_train1.to_numpy()
race_train1_np = race_train1.to_numpy()
for i in range(5):
    acc_i = marginal_accuracy_coef(y_train1, x_train1_np, race_train1_np, i)
    disc_i = marginal_discrimination_coef(y_train1, x_train1_np, race_train1_np, i)
    shapley_acc.append(acc_i)
    shapley_disc.append(disc_i)

names = ['c_charge_degree', 'age_cat', 'sex', 'priors_count', 'length_of_stay']

shapley = pd.DataFrame(list(zip(names, shapley_acc, shapley_disc)),
                          columns=["Feature", "Accuracy",'Discrimination'])
shapley

,Feature,Accuracy,Discrimination
0,c_charge_degree,1.038745,31057.632946
1,age_cat,1.100310,37997.717235
2,sex,0.911121,29300.249982
3,priors_count,1.132691,38543.544855
4,length_of_stay,1.023626,37324.172834


 The obtained outcome demonstrates that 'age_cat' and 'priors_counts' exhibit the most significant influence on accuracy while also serving as strong indicators for discrimination.


In [55]:
def fairness_utility_score(Accuracy, Discr, alpha_value):
    fu_scores = []
    for i in range(5):
        fu_score = Accuracy[i] - alpha_value * Discr[i]
        fu_scores.append(fu_score)
    return fu_scores

In [56]:
alpha1 = pd.DataFrame(list(zip(names, shapley_acc, shapley_disc, fairness_utility_score(shapley['Accuracy'], shapley['Discrimination'], 0.000001))),
                       columns=["Feature", "Accuracy",'Discrimination', 'F_score'])
alpha1

,Feature,Accuracy,Discrimination,F_score
0,c_charge_degree,1.038745,31057.632946,1.007688
1,age_cat,1.100310,37997.717235,1.062313
2,sex,0.911121,29300.249982,0.881820
3,priors_count,1.132691,38543.544855,1.094147
4,length_of_stay,1.023626,37324.172834,0.986302


In [57]:
alpha2 = pd.DataFrame(list(zip(names, shapley_acc, shapley_disc, fairness_utility_score(shapley['Accuracy'], shapley['Discrimination'], 0.00001))),
                       columns=["Feature", "Accuracy",'Discrimination', 'F_score'])
alpha2

,Feature,Accuracy,Discrimination,F_score
0,c_charge_degree,1.038745,31057.632946,0.728169
1,age_cat,1.100310,37997.717235,0.720333
2,sex,0.911121,29300.249982,0.618118
3,priors_count,1.132691,38543.544855,0.747256
4,length_of_stay,1.023626,37324.172834,0.650384


In [58]:
alpha3 = pd.DataFrame(list(zip(names, shapley_acc, shapley_disc, fairness_utility_score(shapley['Accuracy'], shapley['Discrimination'], 0.0001))),
                       columns=["Feature", "Accuracy",'Discrimination', 'F_score'])
alpha3

,Feature,Accuracy,Discrimination,F_score
0,c_charge_degree,1.038745,31057.632946,-2.067018
1,age_cat,1.100310,37997.717235,-2.699461
2,sex,0.911121,29300.249982,-2.018904
3,priors_count,1.132691,38543.544855,-2.721663
4,length_of_stay,1.023626,37324.172834,-2.708791


For the alpha1 =  0.000001, the F-score of 'is_violent_recid' is the lowest.

For the alpha2 = 0.00001, the F-score of 'length_of_stay' is the lowest.

For the alpha3 =  0.0001, the F-score of 'length_of_stay' and 'priors_count' are the lowest. However, due to high marginal accuracy of 'priors_count', it is not considered to be removed.

## 6. Logistic Regression After Shapley Features Selection

#### 6.1.1 Refer to Alpha 1, remove 'sex' which has the lowest F-score.

In [59]:
x_train_subset_vc = x_train1.drop(["sex"],axis = 1)
x_test_subset_vc = x_test1.drop(["sex"],axis = 1)
x_validation_vc = x_validation1.drop(["sex"],axis = 1)

FFS_LogReg_vc = LogisticRegression(random_state = 0).fit(x_train_subset_vc, y_train1)

accuracy_ffs_vc = FFS_LogReg_vc.score(x_validation_vc,y_validation1)
accuracy_ffs_vc

0.7848699763593381

#### 6.1.2 Based on Alpha 2 & 3, remove 'length_of_stay' which has the lowest F-score.

In [60]:
x_train_subset_ls = x_train1.drop(["length_of_stay"],axis = 1)
x_test_subset_ls = x_test1.drop(["length_of_stay"],axis = 1)
x_validation_ls = x_validation1.drop(["length_of_stay"],axis = 1)

FFS_LogReg_ls = LogisticRegression(random_state = 0).fit(x_train_subset_ls, y_train1)

accuracy_ffs_ls = FFS_LogReg_ls.score(x_validation_ls,y_validation1)
accuracy_ffs_ls

0.7919621749408984

## 7. Logistic Regression After calibration Features Selection

In [61]:
def MyCalibration(sensitive_attr, y_pred, y_true):
    cau_index = np.where(sensitive_attr == 1)[0]
    african_index = np.where(sensitive_attr == 0)[0]

    y_pred_cau = y_pred[cau_index]
    y_true_cau = y_true[cau_index]
    Acc_cau = sum(y_pred_cau == y_true_cau)/len(y_pred_cau)

    y_pred_african = y_pred[african_index]
    y_true_african = y_true[african_index]
    Acc_african = sum(y_pred_african == y_true_african)/len(y_pred_african)

    calibration = abs(Acc_cau - Acc_african)
    return(calibration)

In [62]:
Accuracy_lr = []
Calibration_lr = []

for i in ['base'] + features:
    if i == 'base':
        logReg = LogisticRegression(random_state = 0).fit(x_train1, y_train1)
        Accuracy_lr.append(logReg.score(x_test1, y_test1))
        Calibration_lr.append(MyCalibration(race_test1, logReg.predict(x_test1), y_test1))
    else:
        x_train_subset = x_train1.drop([i],axis= 1)
        x_test_subset = x_test1.drop([i],axis = 1)
        logReg = LogisticRegression(random_state = 0).fit(x_train_subset, y_train1)
        Accuracy_lr.append(logReg.score(x_test_subset, y_test1))
        Calibration_lr.append(MyCalibration(race_test1, logReg.predict(x_test_subset), y_test1))

col_names = ['base'] + names
Conclusion_lr = pd.DataFrame(list(zip(col_names, Accuracy_lr, Calibration_lr)),
                          columns=["Eliminating Feature", "Accuracy", "Calibration"])
Conclusion_lr

,Eliminating Feature,Accuracy,Calibration
0,base,0.782506,0.055423
1,c_charge_degree,0.782506,0.065123
2,age_cat,0.784870,0.061041
3,sex,0.782506,0.055423
4,priors_count,0.784870,0.061041
5,length_of_stay,0.782506,0.065123


### By comparing the accuracy of remove 'sex' and 'length of stay' in part 6, model without 'length of stay' has higher accuracy. Therefore the final model is excluded from features: 'length of stay'.

In [63]:
x_train_subset_final = x_train1.drop(["length_of_stay"],axis = 1)
x_test_subset_final = x_test1.drop(["length_of_stay"],axis = 1)
x_validation_final = x_validation1.drop(["length_of_stay"],axis = 1)


FFS_LogReg_final = LogisticRegression(random_state = 0).fit(x_train_subset_final, y_train1)

accuracy_ffs_final = FFS_LogReg_final.score(x_validation_final,y_validation1)
accuracy_ffs_final

0.7919621749408984